In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sjögrens_Syndrome"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Sjögrens_Syndrome/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Sjögrens_Syndrome/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Sjögrens_Syndrome/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Sjögrens_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Search for directories related to Sjögren's Syndrome
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)

# Look for directories related to Sjögren's Syndrome
# Sjögren's might be associated with salivary gland issues or autoimmune responses affecting head and neck
matching_dirs = [dir_name for dir_name in tcga_dirs 
                if any(term in dir_name.lower() for term in 
                       ["sjogren", "sjögren", "salivary", "autoimmune", "head_and_neck"])]

if not matching_dirs:
    print(f"No matching directory found for trait: {trait}")
    # Check if there are any potential related conditions we should explore
    related_conditions = [dir_name for dir_name in tcga_dirs if "head_and_neck" in dir_name.lower()]
    if related_conditions:
        print(f"Potentially related conditions: {related_conditions}")
    
    # Record that this trait is not available and exit
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    print("Task marked as completed. Sjögren's Syndrome is not directly represented in the TCGA dataset.")
else:
    # If we found a matching directory
    print(f"Found matching directories: {matching_dirs}")
    
    # Select the most relevant directory
    selected_dir = matching_dirs[0]
    print(f"Selected directory: {selected_dir}")
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Get file paths for clinical and genetic data
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Step 3: Load the files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # Step 4: Print column names of clinical data
    print("Clinical data columns:")
    print(clinical_df.columns.tolist())


Found matching directories: ['TCGA_Head_and_Neck_Cancer_(HNSC)']
Selected directory: TCGA_Head_and_Neck_Cancer_(HNSC)


Clinical data columns:
['_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_HNSC', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_HNSC', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'alcohol_history_documented', 'amount_of_alcohol_consumption_per_day', 'anatomic_neoplasm_subdivision', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'clinical_M', 'clinical_N', 'clinical_T', 'clinical_stage', 'days_to_additional_surgery_locoregional_procedure', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_completion_of_curative_tx', 'days_to_death', 'days_to_initial_pathologic_d

### Step 2: Find Candidate Demographic Features

In [3]:
# Finding candidate demographic features
candidate_age_cols = ["age_at_initial_pathologic_diagnosis", "days_to_birth"]
candidate_gender_cols = ["gender"]

# Extract and preview candidate columns
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, "TCGA_Head_and_Neck_Cancer_(HNSC)"))
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

print("Age column previews:")
print(age_preview)

# Extract gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("Gender column previews:")
print(gender_preview)


Age column previews:
{'age_at_initial_pathologic_diagnosis': [66.0, 69.0, 49.0, 39.0, 45.0], 'days_to_birth': [-24222.0, -25282.0, -17951.0, -14405.0, -16536.0]}
Gender column previews:
{'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Analyzing candidate columns for demographic information

# For age information, we have two candidates:
# 1. 'age_at_initial_pathologic_diagnosis' - contains actual age values in years
# 2. 'days_to_birth' - contains negative values representing days from birth to diagnosis

# The 'age_at_initial_pathologic_diagnosis' provides age directly in years, which is more 
# intuitive and easier to work with than 'days_to_birth'
age_col = 'age_at_initial_pathologic_diagnosis'

# For gender information, we only have one candidate:
# 'gender' - contains values like 'MALE' and 'FEMALE'
gender_col = 'gender'

# Print information about the chosen columns
print(f"Chosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Use the Head and Neck Cancer directory identified in Step 1
selected_dir = 'TCGA_Head_and_Neck_Cancer_(HNSC)'
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for head and neck cancer patients, which is relevant for studying Sjögren's Syndrome as it often affects salivary glands in the head and neck region."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 604 samples


After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (566, 19851) (samples x features)


After handling missing values, data shape: (566, 19851)
For the feature 'Sjögrens_Syndrome', the least common label is '0' with 44 occurrences. This represents 7.77% of the dataset.
The distribution of the feature 'Sjögrens_Syndrome' in this dataset is fine.

Quartiles for 'Age':
  25%: 53.0
  50% (Median): 61.0
  75%: 68.0
Min: 19.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 151 occurrences. This represents 26.68% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Saved usable linked data to ../../output/preprocess/Sjögrens_Syndrome/TCGA.csv
